### Paquetes

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from tqdm import tqdm as tq
import json

### Funciones

In [2]:
######################################################################################## #EXTRAER LINKS DE LA PÁGINA DE WEB DE LOS DOCUMENTOS
def extraer_links(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        for link in links:
            href = link.get('href')
            if href: 
                None
    else:
        print(f'Error al realizar la petición HTTP: {response.status_code}')
    return links

######################################################################################## #EXTRAER LOS LINKS DE LOS DOCUMENTOS (PDF)
def docs(links, pattern, base):
    docus = []
    for link in links:
        href = link.get('href')
        if href and pattern.match(href):  
            docus.append(base+href)
    docus = list(set(docus))
    return docus

######################################################################################## EXTRAERR TABLA DE LOS COUMENTOS (METADATOS)
def tabla(url):

    # Realiza la petición HTTP a la página
    response = requests.get(url+"?show=full")

    # Verifica que la petición fue exitosa
    if response.status_code == 200:
        # Usa pandas para leer las tablas de la página
        tablas = pd.read_html(response.text)
        tabla = tablas[0]
        tabla.index = tabla[0]; tabla = tabla[[1]]; tabla = tabla.T


        columnas_deseadas = ["dc.creator", "dc.date.issued", "dc.identifier.uri", "dc.description.abstract", "dc.language.iso", "dc.subject", "dc.title", "dc.description.ciudad", "dc.type.spa", "dc.description.programa"]
        columnas_existentes = [col for col in columnas_deseadas if col in tabla.columns]
        tabla_filtrada = tabla[columnas_existentes]

        nombres_columnas = tabla_filtrada.columns.tolist()
        nuevos_nombres_columnas, nombres_vistos = [], {}

        for nombre in nombres_columnas:
            if nombre in nombres_vistos:
                nombres_vistos[nombre] += 1
                nombre_modificado = f"{nombre}_{nombres_vistos[nombre]}"
            else:
                nombres_vistos[nombre] = 0
                nombre_modificado = nombre
            nuevos_nombres_columnas.append(nombre_modificado)

        tabla_filtrada.columns = nuevos_nombres_columnas

    return tabla_filtrada


######################################################################################## DESCARGAR PDFS
def descargar_pdf(url, destino):
    try:
        # Hace una petición GET al URL del PDF
        respuesta = requests.get(url)
        
        # Verifica que la petición fue exitosa (código de estado 200)
        respuesta.raise_for_status()
        
        # Abre el archivo en el modo de escritura binaria para guardar el PDF
        with open(destino, 'wb') as archivo:
            archivo.write(respuesta.content)
            
        print(f"PDF guardado con éxito en: {destino}")
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el PDF: {e}")


### Configuración base

#### Paramétros

In [3]:
# Reemplaza 'url_aqui' con la URL de la página de la que deseas extraer los enlaces
pattern = re.compile(r'^/handle/20\.500\.11761/\d+$')
base = "http://repository.humboldt.org.co"
url = 'http://repository.humboldt.org.co/handle/20.500.11761/31464/browse?rpp=20&sort_by=2&type=dateissued&offset=20&etal=-1&order=ASCC'
links = extraer_links(url)
documentos = docs(links, pattern, base)


#### Links generales: descargar los links de todas las páginas enumeradas de los documentos de investigación

In [17]:
investigacion = []

for i in tq(range(20, 340, 20)):
    # url = f'http://repository.humboldt.org.co/handle/20.500.11761/31464/browse?rpp=20&sort_by=2&type=dateissued&offset={i}&etal=-1&order=ASCC'
    url = f'http://repository.humboldt.org.co/handle/20.500.11761/31468/recent-submissions?offset={i}'
    links = extraer_links(url)
    documentos = docs(links, pattern, base)
    investigacion = investigacion + documentos

inv = pd.DataFrame(data = investigacion, columns = ["link"])

  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:15<00:00,  1.04it/s]


#### Asignar ID a cada documento

In [5]:
def extraer_numero(link):
    match = re.search(r'(\d+)$', link)
    if match:
        return match.group(1)
    return None  

inv['id'] = inv['link'].apply(extraer_numero)

#### Extraer link pdf de cada documento

In [6]:
# documento = extraer_links("http://repository.humboldt.org.co/handle/20.500.11761/32572")
patronpdf = re.compile(r'/bitstream/handle/20\.500\.11761/\d+/.+?\.pdf')
# pdf = docs(documento, patronpdf, base)

pdfs = []

for i in tq(inv["link"]):
    try:
        documento = extraer_links(i)
        pdf = docs(documento, patronpdf, base)
        pdf = pdf[0]
    except:
        pdf = np.nan

    pdfs.append(pdf)


inv["link_pdf"] = pdfs


  0%|          | 1/285 [00:02<09:39,  2.04s/it]

100%|██████████| 285/285 [05:13<00:00,  1.10s/it]


#### Organizar la base

In [9]:
inv = inv[["id", "link", "link_pdf"]]
inv.to_excel("0_LINKS.xlsx", index = False)

<ipython-input-9-f51040d83917>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  inv.to_excel("0_LINKS.xlsx", index = False)


#### Metadatos

In [15]:
inv

,id,link,link_pdf
0,35937,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...
1,35976,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...
2,35945,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...
3,35956,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...
4,35960,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...
...,...,...,...
280,31358,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...
281,31360,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...
282,31361,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...
283,31365,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...


In [10]:
inv = pd.read_excel("0_LINKS.xlsx")

In [11]:
def tabla(url):

    # Realiza la petición HTTP a la página
    response = requests.get(url+"?show=full")

    # Verifica que la petición fue exitosa
    if response.status_code == 200:
        # Usa pandas para leer las tablas de la página
        tablas = pd.read_html(response.text)
        tabla = tablas[0]
        tabla.index = tabla[0]; tabla = tabla[[1]]; tabla = tabla.T


        columnas_deseadas = ["dc.creator", "dc.date.issued", "dc.identifier.uri", "dc.description.abstract", "dc.language.iso", "dc.subject", "dc.title", "dc.description.ciudad", "dc.type.spa", "dc.description.programa"]
        columnas_existentes = [col for col in columnas_deseadas if col in tabla.columns]
        tabla_filtrada = tabla[columnas_existentes]

        nombres_columnas = tabla_filtrada.columns.tolist()
        nuevos_nombres_columnas, nombres_vistos = [], {}

        for nombre in nombres_columnas:
            if nombre in nombres_vistos:
                nombres_vistos[nombre] += 1
                nombre_modificado = f"{nombre}_{nombres_vistos[nombre]}"
            else:
                nombres_vistos[nombre] = 0
                nombre_modificado = nombre
            nuevos_nombres_columnas.append(nombre_modificado)

        tabla_filtrada.columns = nuevos_nombres_columnas

    return tabla_filtrada

In [12]:
metadata = pd.DataFrame()

for id, link in tq(zip(inv.id[:], inv.link[:])):
    try: 
        tablaid = tabla(link)
        tablaid["id"] = id

        metadata = pd.concat([metadata, tablaid], axis = 0)

    except:
        None

metadata = metadata.sort_values(by = ["dc.date.issued"])


285it [05:18,  1.12s/it]


Categoría

In [13]:
def extraer_html(url):
    respuesta = requests.get(url)
    if respuesta.status_code == 200:
        return respuesta.text
    else:
        return "Error en la solicitud, código de estado: " + str(respuesta.status_code)


In [14]:
category = []
for i in tq(metadata["dc.identifier.uri"]):
    html_string = str(extraer_html(i))
    categoria = re.search(r"window\.GTM\.collection='(.*?)';", html_string).group(1)
    category.append(categoria)


100%|██████████| 285/285 [07:06<00:00,  1.50s/it]


In [19]:
metadata.to_excel("1_METADATA.xlsx", index = False)

<ipython-input-19-05dd514adf1e>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  metadata.to_excel("1_METADATA.xlsx", index = False)


#### Descargar PDFs

In [20]:
inv = pd.read_excel("0_LINKS.xlsx")

In [21]:
nod = []

for id, link in tq(zip(inv.id[0:], inv.link_pdf[0:])):
    try:
        descargar_pdf(link, f"PDFS/{id}.pdf")
    
    except:
        nod.append(id)

1it [00:02,  2.83s/it]

PDF guardado con éxito en: PDFS/35937.pdf


2it [00:12,  4.87s/it]

PDF guardado con éxito en: PDFS/35976.pdf


3it [00:18,  5.10s/it]

...

PDF guardado con éxito en: PDFS/35991.pdf


9it [04:29, 40.10s/it]

PDF guardado con éxito en: PDFS/35935.pdf


11it [05:28, 32.65s/it]

PDF guardado con éxito en: PDFS/36015.pdf


#### Leer PDFs

In [1]:
import os
import fitz  # PyMuPDF
import pandas as pd
import glob as glob

In [5]:
# Ruta a la carpeta que contiene los archivos PDF
ruta_carpeta = 'PDFS/*.pdf'
archivos = glob.glob(ruta_carpeta)
datos = []


# Usa glob.glob para encontrar todos los archivos PDF en la carpeta
for ruta_archivo in tq(archivos):

    try:
        # Abre el archivo PDF
        with fitz.open(ruta_archivo) as doc:
            texto = ""
            # Recorre cada página del PDF y extrae el texto
            for pagina in doc:
                texto += pagina.get_text()
        
        # Extrae el nombre del archivo sin la ruta y sin la extensión .pdf
        nombre_archivo_sin_ext = os.path.basename(ruta_archivo)[:-4]
        
        # Añade el nombre y el contenido a la lista
        datos.append({"id": nombre_archivo_sin_ext, "contenido": texto})

    except:
        print("Error")

# Crea un DataFrame con los datos recopilados
texto = pd.DataFrame(datos)



  0%|          | 0/730 [00:00<?, ?it/s]

 16%|█▌        | 115/730 [00:24<01:13,  8.33it/s]mupdf: cannot recognize version marker
mupdf: no objects found


Error


 22%|██▏       | 163/730 [00:45<03:29,  2.71it/s]mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: invalid key in dict
mupdf: no objects found


Error
Error


...

Error


 99%|█████████▉| 723/730 [10:01<00:05,  1.39it/s]mupdf: cannot recognize version marker
mupdf: no objects found


Error


100%|██████████| 730/730 [10:07<00:00,  1.20it/s]

        id                                          contenido
0    31147   \n \n \n \n \n \nHolcim \n \nInstituto de Inv...
1    31152  1 \n \n \n \n \nCONVENIO DE ASOCIACIÓN  08-307...
2    31157  1 \n \nValoración económica de los servicios h...
3    31158   \n \nINFORME FINAL \n \nDOCUMENTO EN REVISION...
4    31159  HORMIGAS \nGUSTAVO PEREZ \n \n \nINTRODUCCIÓN ...
..     ...                                                ...
694   9846   \n \n \n1 \nMEMORIAS INTERCAMBIO DE EXPERIENC...
695   9847   \n \n \nMEMORIAS TALLER DE HERRAMIENTAS DE MA...
696   9848   \n \nCOMPENSACIONES AMBIENTALES EN COLOMBIA \...
697   9852   \n \n1 \nAnálisis comparativo sobre compensac...
698   9875  PLAN DE ACCIÓN REGIONAL EN BIODIVERSIDAD DE LA...

[699 rows x 2 columns]


In [15]:
texto.to_excel("2_TEXTO.xlsx", index = False)

<ipython-input-15-864f80fd42c8>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  texto.to_excel("2_TEXTO.xlsx", index = False)


### Organizar bases

In [38]:
inv = pd.read_excel("0_LINKS.xlsx")
metadata = pd.read_excel("1_METADATA.xlsx")
texto = pd.read_excel("2_TEXTO.xlsx")

In [3]:
# columns_meta = list(metadata.columns)
# columns_meta.sort()

In [4]:
dbf = inv.merge(metadata, "left", "id").merge(texto, "left", "id")

In [5]:
columns_meta = list(metadata.columns)
columns_meta.sort()

In [6]:
import pandas as pd

def combinarc(df, nombre_base):
    # Encuentra todas las columnas que comienzan con el nombre base
    columnas = [col for col in df.columns if col.startswith(nombre_base)]
    columnas.sort(key=lambda x: '' if x == nombre_base else x)
    
    # Inicializa la nueva columna con listas vacías
    df[nombre_base + '_list'] = [[] for _ in range(len(df))]
    
    # Itera sobre cada fila y cada columna relacionada para agregar los valores a la lista
    for index, row in df.iterrows():
        lista_valores = []
        for col in columnas:
            if pd.notnull(row[col]):
                lista_valores.append(row[col])
        df.at[index, nombre_base + '_list'] = lista_valores
    
    # Elimina las columnas originales excepto la columna base
    columnas_a_eliminar = [col for col in columnas if col != nombre_base]
    df.drop(columns=columnas_a_eliminar, inplace=True)
    
    return df

# Asumiendo que 'dbff' es tu DataFrame
# Ejemplo de uso:
# dbff = combinarc(dbff, 'dc.creator')

In [10]:
dbf.to_excel("4_DBF_LONG.xlsx", index = False)

<ipython-input-10-5d9f1261f89e>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  dbf.to_excel("4_DBF_LONG.xlsx", index = False)


In [13]:
dbff = dbf.copy()

In [14]:
repetidas = ['dc.creator', 'dc.description.abstract', 'dc.description.programa', 'dc.subject', 'dc.title', 'dc.type.spa', ]
for variable in repetidas:
    dbff = combinarc(dbff, variable)

In [17]:
dbff.to_excel("5_DBF_SHORT.xlsx", index = False)

<ipython-input-17-1a774c981b16>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  dbff.to_excel("5_DBF_SHORT.xlsx", index = False)


In [18]:
dbff

,id,link,link_pdf,dc.date.issued,dc.identifier.uri,dc.description.abstract,dc.language.iso,dc.subject,dc.title,dc.description.ciudad,...,dc.description.programa,dc.creator,dc.identifier.uri_1,contenido,dc.creator_list,dc.description.abstract_list,dc.description.programa_list,dc.subject_list,dc.title_list,dc.type.spa_list
0,35144,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...,2005-08,http://hdl.handle.net/20.500.11761/35144,En el presente informe se describen las activi...,NaN,Aves,Inventario y monitoreo participativo de las av...,Bogotá,...,Proyecto andes,"Ochoa Quintero, Jose Manuel",NaN,INVENTARIO Y MONITOREO PARTICIPATIVO DE LAS AV...,"[Ochoa Quintero, Jose Manuel, Melo Vasquez, Is...",[En el presente informe se describen las activ...,[Proyecto andes],"[Aves, Censo poblacional, Inventario]",[Inventario y monitoreo participativo de las a...,[]
1,32572,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...,2006-04,http://hdl.handle.net/20.500.11761/32572,Este trabajo de caracterización general sobre ...,spa,Bejucos,Caracterización del uso de bejucos en tres zon...,Bogotá,...,Proyecto Andes,"Martínez Bedoya, Yerly Andrés",NaN,INSTITUTO DE INVESTIGACION DE RECURSOS BIOLOGI...,"[Martínez Bedoya, Yerly Andrés, Alvarez Rodas,...",[Este trabajo de caracterización general sobre...,[Proyecto Andes],"[Bejucos, usos, Artesanos, sostenibilidad]",[Caracterización del uso de bejucos en tres zo...,[]
2,9317,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...,2007-12-28,http://hdl.handle.net/20.500.11761/9317,La caracterización local de la vegetación repr...,spa,Flora,Caracterización florística en diferentes cober...,"(Bogotá, Colombia)",...,Paisajes Rurales,"Méndez, Enrique",NaN,CARACTERIZACIÓN FLORÍSTICA EN DIFERENTES \nCOB...,"[Méndez, Enrique]",[La caracterización local de la vegetación rep...,[Paisajes Rurales],"[Flora, Vegetación, reserva forestal, Conserva...",[Caracterización florística en diferentes cobe...,[informe]
3,9714,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...,2006,http://hdl.handle.net/20.500.11761/9714,"Al inicio del desarrollo de este documento, la...",spa,Recursos genéticos,Acuerdos contractuales de acceso a recursos ge...,Bogotá,...,Política y legislación,"Vélez Llinas, Juliana",NaN,NaN,"[Vélez Llinas, Juliana]","[Al inicio del desarrollo de este documento, l...",[Política y legislación],"[Recursos genéticos, productos derivados, prop...",[Acuerdos contractuales de acceso a recursos g...,[informe]
4,34986,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...,2006-02,http://hdl.handle.net/20.500.11761/34986,Las áreas de Importancia para la conservación ...,spa,Avifauna,Conservación de la avifauna en el Aica la pata...,Armenia,...,Proyecto Andes,NaN,NaN,"... . , ... , . \n"" . \n:;:;�j¡;!�lúAvJ�a.:�j�...",[],[Las áreas de Importancia para la conservación...,[Proyecto Andes],"[Avifauna, Habitat, Especies migratorias, Educ...",[Conservación de la avifauna en el Aica la pat...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,36153,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...,2023,http://hdl.handle.net/20.500.11761/36153,Ante la contingencia presentada por el afloram...,spa,Evaluación del impacto ambiental,Entregable 2.5. Informe parcial con los result...,Bogotá D.C.,...,NaN,"Díaz Gómez, Diana Lucía",NaN,\n1 \n \n \n \n \n \n \nCONTRATO DE PRESTACIÓ...,"[Díaz Gómez, Diana Lucía, Valdivieso Camacho, ...",[Ante la contingencia presentada por el aflora...,[],"[Evaluación del impacto ambiental, Monitoreo d...",[Entregable 2.5. Informe parcial con los resul...,[informe]
739,36274,http://repository.humboldt.org.co/handle/20.50...,http://repository.humboldt.org.co/bitstream/ha...,2023,http: